In [ ]:
import xarray as xr

import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('default')
sns.set_palette("colorblind")
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.weight'] = 'light'
rcParams['mathtext.fontset'] = 'cm'
rcParams['mathtext.rm'] = 'serif'
mpl.rcParams["figure.dpi"] = 500
import cartopy.crs as ccrs
import cartopy as ct
import matplotlib.colors as c
import regionmask
import cmasher as cmr
import scipy
from cartopy.util import add_cyclic_point
mpl.rcParams['hatch.linewidth'] = 0.375
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from geocat.comp import eofunc_eofs, eofunc_pcs
from datetime import datetime
import warnings
from matplotlib.patches import Rectangle
import pdo_functions
import importlib
importlib.reload(pdo_functions)
import random
import numba

In [ ]:
cape_1940s = xr.open_dataset('/hurrell-scratch/ivyglade/pdo/ERA5_cape/ERA5_cape/1940s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19400101-19491231.nc')['CAPE'][184*10:]
cape_1950s = xr.open_dataset('/hurrell-scratch/ivyglade/pdo/ERA5_cape/ERA5_cape/1950s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19500101-19591231.nc')['CAPE'][184*10:]
cape_1960s = xr.open_dataset('/hurrell-scratch/ivyglade/pdo/ERA5_cape/ERA5_cape/1960s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19600101-19691231.nc')['CAPE'][184*10:]
cape_1970s = xr.open_dataset('/hurrell-scratch/ivyglade/pdo/ERA5_cape/ERA5_cape/1970s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19700101-19791231.nc')['CAPE'][184*10:]
cape_1980s = xr.open_dataset('/hurrell-scratch/ivyglade/pdo/ERA5_cape/ERA5_cape/1980s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19800101-19891231.nc')['CAPE']
cape_1990s = xr.open_dataset('/hurrell-scratch/ivyglade/pdo/ERA5_cape/ERA5_cape/1990s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.19900101-19991231.nc')['CAPE']
cape_2000s = xr.open_dataset('/hurrell-scratch/ivyglade/pdo/ERA5_cape/ERA5_cape/2000s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.20000101-20091231.nc')['CAPE']
cape_2010s = xr.open_dataset('/hurrell-scratch/ivyglade/pdo/ERA5_cape/ERA5_cape/2010s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.20100101-20191231.nc')['CAPE']
cape_2020s = xr.open_dataset('/hurrell-scratch/ivyglade/pdo/ERA5_cape/ERA5_cape/2020s/daily_max/e5.oper.an.sfc.128_059_cape.daily_max.20200101-20241130.nc')['CAPE']

In [ ]:
cape_1940s_conus = cape_1940s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_1950s_conus = cape_1950s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_1960s_conus = cape_1960s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_1970s_conus = cape_1970s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_1980s_conus = cape_1980s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_1990s_conus = cape_1990s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_2000s_conus = cape_2000s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_2010s_conus = cape_2010s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))
cape_2020s_conus = cape_2020s.sel(longitude=slice(pdo_functions.convert_longitudes(-125), pdo_functions.convert_longitudes(-66))).sel(latitude=slice(49, 24))

In [ ]:
# Concatenate time periods together
cape_conus = xr.concat([cape_1940s_conus, cape_1950s_conus, cape_1960s_conus, cape_1970s_conus, cape_1980s_conus, cape_1990s_conus, cape_2000s_conus,\
                        cape_2010s_conus, cape_2020s_conus], dim='time')

In [ ]:
# Select MAM only and also resample by month
cape_conus_mam = cape_conus.sel(time=cape_conus.time.dt.month.isin([3, 4, 5])).resample(time='1M').mean().dropna(dim='time')

In [ ]:
# Subsetting the southeast rectangle to compare detrend methods
cape_se_mam = cape_conus_mam.sel(longitude=slice(pdo_functions.convert_longitudes(-102), pdo_functions.convert_longitudes(-82))).sel(latitude=slice(38, 30))

In [ ]:
# Making a domain plot so that we can take a regional subset
fig, ax = plt.subplots(subplot_kw=dict(projection=ccrs.AlbersEqualArea(central_longitude=-96, central_latitude=37.5)))

ax.coastlines(lw=0.25, color='xkcd:gunmetal')
ax.add_feature(ct.feature.STATES, lw=0.25, edgecolor='xkcd:gunmetal')
ax.spines['geo'].set_linewidth(0.25)
ax.spines['geo'].set_edgecolor('xkcd:gunmetal')

ax.pcolormesh(cape_conus['longitude'], cape_conus['latitude'], cape_conus_mam.mean(dim='time'), transform=ccrs.PlateCarree(), shading='auto', cmap='Reds')
# ax.pcolormesh(cape_se_mam['longitude'], cape_se_mam['latitude'], cape_se_mam.mean(dim='time'), transform=ccrs.PlateCarree())
rect = Rectangle((-102, 30), -82 - -102, 38-30, transform=ccrs.PlateCarree(), facecolor='none', edgecolor='black', linewidth=2)

ax.add_patch(rect)

In [ ]:
def detrend_dim(da, dim, deg):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit

In [ ]:
lin_detrend = detrend_dim(cape_se_mam, 'time', 1)
poly_detrend = detrend_dim(cape_se_mam, 'time', 2)

In [ ]:
# Remove the seasonal cycle
lin_detrend_no_climo = lin_detrend.groupby('time.month') - lin_detrend.groupby('time.month').mean()
poly_detrend_no_climo = poly_detrend.groupby('time.month') - poly_detrend.groupby('time.month').mean()
cape_se_mam_no_climo = cape_se_mam.groupby('time.month') - cape_se_mam.groupby('time.month').mean()

In [ ]:
# Create weights, apply area weighting and take area average
weights = np.cos(np.deg2rad(lin_detrend_no_climo.latitude))

lin_detrend_no_climo_mean = lin_detrend_no_climo.weighted(weights).mean(dim=('latitude', 'longitude'))
poly_detrend_no_climo_mean = poly_detrend_no_climo.weighted(weights).mean(dim=('latitude', 'longitude'))
cape_se_mam_no_climo_mean = cape_se_mam_no_climo.weighted(weights).mean(dim=('latitude', 'longitude'))

In [ ]:
255/3

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

years = np.arange(1940, 2025, 1)

ax.set_xlim(1940, 2024)

ax.axhline(0, 1940, 2024, linestyle='--', color='xkcd:gunmetal', lw=0.5)

ax.plot(years, cape_se_mam_no_climo_mean.groupby('time.year').mean(), label='no detrend', color='black')
ax.plot(years, lin_detrend_no_climo_mean.groupby('time.year').mean(), label='linear detrend', color='xkcd:dull blue')
ax.plot(years, poly_detrend_no_climo_mean.groupby('time.year').mean(), label='degree 2 polynomial detrend', color='xkcd:avocado green')

ax.legend(frameon=False, labelspacing=0.5)

ax.set_ylim(-350, 450)
ax.set_ylabel('J kg$^{-1}$')

ax.set_title('Southeast US MAM Annual Mean CAPE Anomalies')

ax.set_xlabel('Year')